# Load Library & Preprocess Data

In [1]:
import os
GPU = f'0'
os.environ['CUDA_VISIBLE_DEVICES']=GPU
import random
from glob import glob
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import (
    ReduceLROnPlateau,
    EarlyStopping,
    ModelCheckpoint,
    TensorBoard
)
from tensorflow.keras.applications import (
    MobileNet,
    MobileNetV2,
    EfficientNetB7
)
from tensorflow.keras.layers import (
    GlobalAveragePooling2D,
    Dense
)
from sklearn.model_selection import train_test_split

#### Set Seed 

In [2]:
SEED = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

seed_everything(SEED)

#### Config 

In [3]:
batch_size = 1

classes = 1049
size = 600

#### Load data 

In [4]:
df = pd.read_csv('train.csv')
df.head()

,id,landmark_id
0,../../uiui/datasets/public/train/경기도/동탄 복합문화센터...,114
1,../../uiui/datasets/public/train/경기도/동탄 복합문화센터...,114
2,../../uiui/datasets/public/train/경기도/동탄 복합문화센터...,114
3,../../uiui/datasets/public/train/경기도/동탄 복합문화센터...,114
4,../../uiui/datasets/public/train/경기도/동탄 복합문화센터...,114


In [5]:
df['landmark_id'] = df['landmark_id'].astype(str)

In [6]:
datagen = ImageDataGenerator(
    rescale=1./255,
)
generator = datagen.flow_from_dataframe(
    df,
    x_col='id',
    y_col='landmark_id',
    target_size=(size,size),
    batch_size=batch_size,
    shuffle=False
)

Found 88102 validated image filenames belonging to 1049 classes.


# Load Model

In [7]:
with tf.device(f'/device:GPU:{GPU}'):
    base_model = EfficientNetB7(
        input_shape=(size,size,3),
        include_top=False,
        weights='imagenet',
    )
    base_model.trainable = False
    
    model = tf.keras.Sequential([
        base_model,
        GlobalAveragePooling2D(),
    ])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb7 (Functional)  (None, 19, 19, 2560)      64097687  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2560)              0         
Total params: 64,097,687
Trainable params: 0
Non-trainable params: 64,097,687
_________________________________________________________________


In [8]:
features = []
targets = []

for data in tqdm(generator):
    features.append(model(data[0]).numpy().reshape([-1]).tolist())
    targets.append(np.argmax(data[1], axis=1)[0])

  0%|          | 9/88102 [00:04<11:47:42,  2.07it/s]


In [9]:
len(features), len(targets)

(10, 10)

In [10]:
feature_df = pd.DataFrame(features)
target_df = pd.DataFrame(targets)
target_df.columns = ['label']

In [11]:
final = pd.concat([feature_df, target_df], axis=1)

In [12]:
final.to_csv('train_features.csv', index=False)

In [13]:
pd.read_csv('train_features.csv')

,0,1,2,3,4,5,6,7,8,9,...,2551,2552,2553,2554,2555,2556,2557,2558,2559,label
0,-0.033111,-0.087834,-0.161715,0.093728,-0.055821,-0.065919,-0.131588,-0.137830,-0.130928,-0.139730,...,-0.165677,-0.168668,-0.193857,-0.130095,-0.184072,-0.127768,-0.162794,-0.147641,-0.114176,67
1,-0.028617,-0.086569,-0.160131,0.105065,-0.054442,-0.063386,-0.132863,-0.134234,-0.128608,-0.139161,...,-0.165698,-0.168344,-0.192635,-0.128562,-0.183626,-0.127284,-0.164049,-0.147185,-0.112684,67
2,-0.027658,-0.085857,-0.160032,0.107067,-0.055544,-0.063257,-0.132713,-0.134500,-0.129093,-0.139190,...,-0.166109,-0.168516,-0.192889,-0.128501,-0.183164,-0.127530,-0.164386,-0.147436,-0.112076,67
3,-0.027509,-0.085877,-0.160238,0.106865,-0.055520,-0.063339,-0.132712,-0.134615,-0.129154,-0.139242,...,-0.166172,-0.168452,-0.192833,-0.128403,-0.183206,-0.127404,-0.164384,-0.147398,-0.112089,67
4,-0.033413,-0.087631,-0.161997,0.092883,-0.056311,-0.066062,-0.131458,-0.137635,-0.130834,-0.139807,...,-0.165565,-0.168686,-0.193903,-0.130181,-0.184134,-0.127396,-0.162546,-0.147520,-0.114279,67
5,-0.031215,-0.087596,-0.161338,0.097466,-0.055691,-0.065350,-0.132266,-0.136704,-0.129750,-0.139420,...,-0.165911,-0.168480,-0.193420,-0.129438,-0.183894,-0.127366,-0.163085,-0.147493,-0.113592,67
6,-0.032162,-0.087445,-0.161725,0.096043,-0.056222,-0.065485,-0.131846,-0.137258,-0.130311,-0.139666,...,-0.165865,-0.168658,-0.193782,-0.130010,-0.184093,-0.127404,-0.162949,-0.147591,-0.113798,67
7,-0.032361,-0.087730,-0.161791,0.093945,-0.055590,-0.065511,-0.131731,-0.137316,-0.130622,-0.139674,...,-0.165637,-0.168594,-0.193545,-0.129880,-0.183968,-0.127582,-0.162831,-0.147559,-0.114155,67
8,-0.027681,-0.086726,-0.160581,0.106862,-0.054773,-0.063610,-0.133301,-0.134038,-0.128321,-0.139152,...,-0.165905,-0.168371,-0.192381,-0.128401,-0.183603,-0.126803,-0.164205,-0.147244,-0.112430,67
9,-0.030078,-0.087239,-0.161065,0.102373,-0.055163,-0.065062,-0.132958,-0.135942,-0.129451,-0.139331,...,-0.165952,-0.168462,-0.193076,-0.129274,-0.184091,-0.127084,-0.163853,-0.147529,-0.113240,67
